In [1]:
# -*- coding: utf-8 -*-
import random
import torch
from torch.autograd import Variable

In [2]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


In [3]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

In [6]:
# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


0 4.778768526725319e-12
1 4.767222207269217e-12
2 4.848681352254136e-12
3 4.849004878182406e-12
4 4.532160237380456e-12
5 4.836716097078586e-12
6 4.61679479368815e-12
7 5.022692331491108e-12
8 5.01188153478882e-12
9 4.7107222636566526e-12
10 5.07474791355822e-12
11 5.56850572597245e-12
12 5.011225809314901e-12
13 5.025898100474713e-12
14 4.9707885377281436e-12
15 5.284829865392915e-12
16 5.928872844063182e-12
17 6.060399578011744e-12
18 5.582084273980659e-12
19 5.149361839706934e-12
20 5.003789917135126e-12
21 5.112714071553448e-12
22 5.0527715692028075e-12
23 5.073795116689039e-12
24 5.4790993790632125e-12
25 5.301401245078052e-12
26 5.439964017445176e-12
27 5.709054323038698e-12
28 5.8957868968867455e-12
29 5.868144511977924e-12
30 5.793609082066498e-12
31 4.859699448411803e-12
32 5.068386248890944e-12
33 5.238660200079792e-12
34 6.061204056023728e-12
35 5.647856748253188e-12
36 6.001800185312378e-12
37 6.153278841319887e-12
38 5.250241647686282e-12
39 5.127148271916182e-12
40 5.4496

370 5.430162829805907e-12
371 5.847298339967111e-12
372 5.833361571561113e-12
373 5.658917778816885e-12
374 5.463878047923254e-12
375 5.377594203270775e-12
376 5.4796302044468614e-12
377 6.00264456196431e-12
378 5.205001794156283e-12
379 4.91865185733853e-12
380 5.073606899191896e-12
381 5.3044612972896754e-12
382 5.309893583854697e-12
383 5.3567423934086644e-12
384 5.06942274616784e-12
385 5.448877026664745e-12
386 5.142507513572481e-12
387 5.019394188482407e-12
388 4.560667381942052e-12
389 4.761333254749145e-12
390 5.182100842188175e-12
391 5.099002817199327e-12
392 4.82423172590285e-12
393 5.275636698331976e-12
394 5.207657222117135e-12
395 5.326162254293276e-12
396 5.208677239521009e-12
397 5.311021587794951e-12
398 4.977739574696383e-12
399 5.365672316182124e-12
400 4.995537837559905e-12
401 5.167532200756053e-12
402 4.531191394319123e-12
403 4.93079795743645e-12
404 5.345203012846467e-12
405 4.992196326464304e-12
406 5.16081578513794e-12
407 5.441496645636201e-12
408 4.902112136